In [ ]:
import os
import autogen
from textwrap import dedent
import sys

sys.path.append(os.path.abspath(".."))  # Add the parent directory
from finrobot.utils import register_keys_from_json
from finrobot.agents.workflow import SingleAssistantShadow

In [ ]:
llm_config = {
    "config_list": autogen.config_list_from_json(
        "../OAI_CONFIG_LIST",
        filter_dict={
            "model": ["llama-3.3-70b-versatile"], #Change model according to requirement
        },
    ),
    "timeout": 120,
    "temperature": 0.5,
}
register_keys_from_json("../config_api_keys")

In [ ]:
work_dir = "weather_report"
os.makedirs(work_dir, exist_ok=True)
assistant = SingleAssistantShadow(
    "Weather_Analyst",
    llm_config,
    max_consecutive_auto_reply=None,
    human_input_mode="TERMINATE"
)

In [ ]:
location= "Nagpur, Maharashtra"

message = dedent(
    f"""
    Using the tools at your disposal, generate a comprehensive weather analysis report for **{location}**, considering the year **2020**. The report should focus on **rainfall and temperature data**, analyzing their impact on agriculture and predicting **suitable crops for cultivation**. Support your recommendations with detailed reasoning and evidence. 
    Pay attention to the following:
    - Explicitly explain your working plan before starting.
    - Use tools step-by-step to gather historical monthly rainfall and temperature data for {location} for 2020.
    - Using the provided function gather soil information, crop information (Suggest atleast two crops and gather info) and information about crop-soil compatibility.
    - Store all files in **"{work_dir}"** and ensure the data is accurate.
    
    After gathering data of monthly average rainfall, temperature and crop-soil related information, analyze it (data gathered will in in JSON format), build comprehensive report by adding facts and numerical data (wherever necessary) from gathered data and Return the final report as a JSON object following this format:
    {{
        "Analysis_of_Rainfall_Trends": "...",
        "Analysis_of_Temperature": "...",
        "Crop_Suitability_and_Crop_Growth_Requirements": "...",
        "Predictive_Insights_of_weather": "..."
    }}
    Ensure each field is to be filled with well-reasoned, paragraph-style content.
    Each key must have a **detailed paragraph** as the value (approx. 180–220 words per section) explaining the relevant analysis and insights. Avoid bullet points; use well-structured natural language paragraphs.
    
    After this, use the `build_structured_analysis_report` tool to convert the JSON into a readable PDF report.
    """
)


assistant.chat(message, use_cache=True, max_turns=50,
               summary_method="last_msg")